In [1]:
from modules.scouter_handler import ScouterHandler
from modules.doc_graph_analyzer import DocGraphAnalyzer
from modules.multi_doc_summary import MultiDocSummary
from modules.utils import make_morp_sentence_list

Loading model parameters.
IO line:45 get_fileds, datatype text
model line:127 self.no_pack_padded_seq False
ModelConstructor line:191, model.obj_f
ml
ModelConstructor line:195, tgt vocab len 50004
ModelConstructor line:196, tgt vocab freq len 177439
Loading model parameters.


In [2]:
keyword = "다스 대통령"

min_rel_score = 2.0
max_length = 3000

In [3]:
scouter = ScouterHandler(size=1000)

In [4]:
query_body = scouter.make_keyword_query_body(keyword, filters=['news_id', 'node', 'edge', 'extContent'])
doc_info_list = scouter.search(query_body, data_type="graphs", trim_lower=True)

Query : 다스 대통령
Scroll idx : 1 (268 docs)
Trim lower 70% of docs (80 docs are cut)
Total retrieved Doc # :  187



In [5]:
doc_info_list = [d for d in doc_info_list if d['rel_score'] > min_rel_score and len(d['extContent']) < max_length]
num_doc = len(doc_info_list)
num_doc

186

In [6]:
doc_graph_anal = DocGraphAnalyzer(keyword)
doc_graph_anal.process_data(doc_info_list, top_doc_num=5, top_keyword_num=5)

Doc #  : 186
Node # : 1003
Edge # : 35958

Community 0 :  135 Keyword, (구속/지난/국장/재산/조성)
Community 1 :  162 Keyword, (회장/진술/도곡동/매각/대금)
Community 2 :   71 Keyword, (정부/위원/영향/측근/방송)
Community 3 :  170 Keyword, (청와대/당시/주장/기록/변호)
Community 4 :  173 Keyword, (수수/불법/뇌물/의원/공천)
Community 5 :  127 Keyword, (지검/서울중앙/통보/기자/검사)
Community 7 :  155 Keyword, (관계/입장/진행/사건/전직)



In [7]:
top_doc_id_query_list = [scouter.make_doc_id_query_body(top_doc_ids, filters=["news_id", "newsTitle", "extContent", "analyzed_text"])
                         for top_doc_ids in doc_graph_anal.top_doc_id_list]
top_doc_info_list = [scouter.search(doc_id_query, data_type='newspaper', trim_lower=False, silence=True)
                     for doc_id_query in top_doc_id_query_list]

In [8]:
top_doc_info_list[0][0]['extContent']

'서울중앙지검 첨단범죄수사1부(신봉수 부장검사)는 지난 2일 이 국장을 특정경제범죄가중처벌법상 횡령과 배임 및 증거인멸 혐의 등으로 구속기소 했다고 4일 밝혔다. 검찰은 이 국장의 구속영장에 이어 공소장에도 이 전 대통령을  다스의 실소유주 라고 명시한 것으로 전해졌다. 이 국장은 검찰 수사에 대비해 자신이 관리하던 입출금 장부 등을 파기한 혐의로 지난달 13일 긴급체포된 후 구속됐다. 검찰은 이 전 대통령 측이 이 장부에 차명재산을 정리해 관리해온 것으로 의심한다. 이 국장도 체포 이후 이를 시인한 것으로 전해졌다. 검찰은 이 국장을 조사하는 과정에서 그가 2009년∼2013년 다스 자회사 홍은프레닝에서 10억8천만원, 2009년 다스 관계사 금강에서 8억원을 각각 횡령한 혐의를 확인했다. 그는 지난해 12월 홍은프레닝을 통해 이 전 대통령의 아들 이시형 다스 전무가 장악한 관계사 다온에 40억원 가량을 무담보 저리로 특혜 대출해 준 배임 혐의도 있다. 배임 혐의와 관련해 검찰은 이 전무가 자금 지원을 이 국장에게 지시했다며  공범 으로 공소장에 적시한 것으로 전해졌다. 검찰은 이 국장이 횡령 등으로 조성한 자금이 이 전 대통령의 비자금처럼 쓰인 것이 아닌지 의심해 구체적인 사용처 등 돈의 흐름을 추적하고 있다. 다스 실소유주 등 이 전 대통령 차명재산 의혹의 열쇠를 쥔 인물로 평가받는 이 국장이 재판에 넘겨짐에 따라 법조계에서는 이 전 대통령에 대한 검찰 조사가 임박했다는 관측이 나온다. 서울중앙지검 윤석열 검사장은 이르면 5∼6일께 이 전 대통령에 대한 직접 조사 계획을 문무일 검찰총장에게 보고할 것으로 알려졌다.'

In [9]:
top_doc_morph_sentence_list = [sum([make_morp_sentence_list(doc_info['analyzed_text'])
                                    for doc_info in doc_info_list], []) for doc_info_list in top_doc_info_list]
top_keyword_list = doc_graph_anal.top_keyword_list

In [10]:
multi_doc_summ = MultiDocSummary(keyword)

In [11]:
multi_doc_summ.process_data(top_doc_morph_sentence_list, top_keyword_list)

feature_names_length =  1992
paraent pid 839


In [17]:
multi_doc_summ.main_json[0]

'서울중앙지검 첨단범죄수사 1 부 ( 부장검사 화 ) 는 다스 비자금 운용에 관여한 혐의 ( 특정경제범죄가중처벌법상 횡령 ) 등으로 구속영장을 신청할 방침인 것으로 알려졌다 . 검찰은 이 전 대통령의 구속영장 청구 가능성을 배제하지 않은 채 막바지 보강 수사를 진행 중이다 .'

In [16]:
query_body = scouter.make_keyword_query_body(keyword)
doc_info_list = scouter.search(query_body, data_type="summary_by_time", trim_lower=True)
doc_save_list = preprocess(doc_info_list, len(doc_info_list))

GET http://155.230.34.154:9200/summary_by_time/text/_search?scroll=10m&size=1000 [status:404 request:0.003s]


Query : 다스 대통령


NotFoundError: NotFoundError(404, 'index_not_found_exception', 'no such index')